In [75]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

from scipy.stats import spearmanr
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


In [3]:
%cd ..

/home/jan/misc/copenhagen-hack/optimal-ph/aux/model-playground


In [4]:
train_meta = pd.read_csv("../train.csv", index_col=0)
valid_meta = pd.read_csv("../valid.csv", index_col=0)
train_meta

,mean_growth_PH,sequence,representative,is7
99973,7.40,MEANHGMNNYIKLAFVFGITTMATSYADTVAPPTLLTAQKLPQLQQ...,True,False
84793,7.00,MEFFKKTALAALVMGFSGAALALPNITILATGGTIAGGGDSATKSN...,False,True
27864,6.50,MSPLGILRRHRVAALLGAALIISPVVVSFAQSANSTGVSKIVATTQ...,True,False
46228,7.80,MKKQYWYVIITYVAMQLSSLVGVPLLAHSGFINASNKDIAISIASG...,False,False
6028,7.00,MSKKKMAITLSAMLSATIIPSFTMDVHAEKKEETKNTKIELENGMT...,False,True
...,...,...,...,...
69002,7.00,MEIIMRNLCFLLTLVATLLLHGRLIAAALPQDEKLITGQLDNGLRY...,False,True
75674,7.00,MSKHPKLLVLALACLACAGRASAAPASDEVARLAQRCAPDVSPLTM...,False,True
50377,7.45,MSRAGSLMLVLGTALWLCGCSGMNSENKRVAPVAEKRPHTMSLHGV...,True,False
87875,7.00,MSAGRLNKKSLGIVMLLSVGLLLAGCSGSKSSDTGTYSGSVYTVKR...,False,True


In [5]:
freq1 = pd.read_csv("data/1-mers.tsv", sep="\t")
freq2 = pd.read_parquet("data/2-mers.parquet.gzip")
phychem = pd.read_csv("../../data/physchem/properties.csv", index_col=0).drop("ID", axis=1).reset_index(drop=True)


In [70]:
correlations = pd.concat([
    pd.read_csv("data/correlationonemer.csv", index_col=0).rename({"onemer_name": "feature"}, axis=1),
    pd.read_csv("data/correlationtwomer.csv", index_col=0).rename({"twomer_name": "feature"}, axis=1),
    pd.read_csv("data/correlationsprot.csv", index_col=0).rename({"prop_name": "feature"}, axis=1),
], ignore_index=True)

correlations
correlations = correlations[correlations.feature.notna()]
correlations["corr_abs"] = correlations.correlation.abs()
correlations = correlations.sort_values("corr_abs", ascending=False)

In [71]:
features = pd.concat([freq1, freq2, phychem], axis=1)
features

,A,C,D,E,F,G,H,I,K,L,...,YS,YT,YV,YW,YY,isoelectricity,length,hydrophobicity,weight,aliphatic
0,0.069395,0.008897,0.081851,0.055160,0.039146,0.083630,0.014235,0.062278,0.087189,0.072954,...,0.005348,0.000000,0.001783,0.001783,0.016043,4.860335,562,-0.510676,62824.47114,72.064057
1,0.100739,0.003358,0.069174,0.046340,0.034923,0.089993,0.006716,0.040967,0.076561,0.058428,...,0.002688,0.006720,0.004032,0.000672,0.005376,5.032207,1489,-0.343586,160513.00714,73.183345
2,0.090598,0.008547,0.046154,0.015385,0.034188,0.099145,0.006838,0.058120,0.047863,0.068376,...,0.006849,0.006849,0.001712,0.001712,0.000000,7.378459,585,-0.076068,61639.23834,75.743590
3,0.102216,0.000715,0.064332,0.038599,0.027162,0.071480,0.010007,0.060043,0.064332,0.075768,...,0.007868,0.003577,0.004292,0.000000,0.000000,4.722769,1399,-0.150322,147316.17644,84.953538
4,0.117509,0.008226,0.059929,0.023502,0.024677,0.088132,0.015276,0.041128,0.041128,0.057579,...,0.002353,0.005882,0.003529,0.000000,0.001176,5.476988,851,-0.269330,90706.29954,69.330200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104996,0.125954,0.007634,0.061069,0.022901,0.061069,0.099237,0.019084,0.030534,0.034351,0.068702,...,0.003831,0.000000,0.003831,0.000000,0.000000,10.461661,262,-0.385496,28757.43674,62.366412
104997,0.140044,0.000000,0.061269,0.070022,0.028446,0.089716,0.013129,0.037199,0.006565,0.096280,...,0.000000,0.000000,0.002193,0.000000,0.000000,4.959857,457,-0.029103,48805.45654,94.617068
104998,0.086253,0.000000,0.075472,0.061995,0.035040,0.110512,0.013477,0.067385,0.032345,0.075472,...,0.000000,0.000000,0.000000,0.000000,0.000000,4.262404,371,-0.009164,38776.69354,92.479784
104999,0.146667,0.015238,0.076190,0.043810,0.032381,0.102857,0.030476,0.064762,0.026667,0.074286,...,0.007634,0.000000,0.000000,0.000000,0.000000,5.295376,525,-0.120381,55572.00124,83.809524


In [72]:
TOP_N = 100
top_features = correlations[:TOP_N].feature
features = features[top_features]

In [79]:
def get_xy(metadata, features, only_repr = False, only_not7 = False):
    if only_repr:
        metadata = metadata[metadata.representative]
    if only_not7:
        metadata = metadata[~metadata.is7]
    
    y = metadata.mean_growth_PH.to_numpy()
    X = features.loc[metadata.index].to_numpy()
    return X, y

X_train, y_train = get_xy(train_meta, features, only_not7=True)
X_valid, y_valid = get_xy(valid_meta, features, only_not7=True)

scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_valid = scaler.transform(X_valid)

pca = PCA(n_components = 25).fit(X_train)
X_train = pca.transform(X_train)
X_valid = pca.transform(X_valid)

In [80]:
X_train.shape, y_train.shape

((28168, 25), (28168,))

In [81]:
X_valid.shape, y_valid.shape

((7042, 25), (7042,))

In [82]:
X_train_repr, y_train_repr = get_xy(train_meta, features, only_not7=True, only_repr=True)
X_valid_repr, y_valid_repr = get_xy(valid_meta, features, only_not7=True, only_repr=True)

X_train_repr = pca.transform(scaler.transform(X_train_repr))
X_valid_repr = pca.transform(scaler.transform(X_valid_repr))


X_train_repr.shape, X_valid_repr.shape

((14209, 25), (3552, 25))

In [83]:
models = [
    ("Ridge a0.5", Ridge(alpha=0.5, random_state = 31415)),
    ("Ridge a1", Ridge(alpha=1.0, random_state = 31415)),
    ("Ridge a2", Ridge(alpha=2.0, random_state = 31415)),
    ("RandomForest d4", RandomForestRegressor(max_depth=4, random_state=31415)),

]

for name, model in models:
    model.fit(X_train, y_train)
    train_pred = model.predict(X_train)
    valid_pred = model.predict(X_valid)
    valid_repr_pred = model.predict(X_valid_repr)


    print(">>", name)
    print("Train | Spearman {:.4f} RMSE {:.4f}".format(spearmanr(train_pred, y_train)[0], np.sqrt(mean_squared_error(train_pred, y_train))))
    print("Valid | Spearman {:.4f} RMSE {:.4f}".format(spearmanr(valid_pred, y_valid)[0], np.sqrt(mean_squared_error(valid_pred, y_valid))))
    print("ValRe | Spearman {:.4f} RMSE {:.4f}".format(spearmanr(valid_repr_pred, y_valid_repr)[0], np.sqrt(mean_squared_error(valid_repr_pred, y_valid_repr))))

>> Ridge a0.5
Train | Spearman 0.2979 RMSE 1.0301
Valid | Spearman 0.2708 RMSE 1.0190
ValRe | Spearman 0.3116 RMSE 1.1324
>> Ridge a1
Train | Spearman 0.2979 RMSE 1.0301
Valid | Spearman 0.2708 RMSE 1.0190
ValRe | Spearman 0.3116 RMSE 1.1324
>> Ridge a2
Train | Spearman 0.2979 RMSE 1.0301
Valid | Spearman 0.2708 RMSE 1.0190
ValRe | Spearman 0.3116 RMSE 1.1324
>> RandomForest d4
Train | Spearman 0.3297 RMSE 1.0140
Valid | Spearman 0.2919 RMSE 1.0046
ValRe | Spearman 0.2966 RMSE 1.1268


In [84]:
models = [
    ("SVR C10", SVR(C=10.0, epsilon=0.1)),
]

for name, model in models:
    model.fit(X_train_repr, y_train_repr)
    print("fit!")
    train_pred = model.predict(X_train_repr)
    valid_pred = model.predict(X_valid)
    valid_repr_pred = model.predict(X_valid_repr)

    print(">>", name)
    print("Train | Spearman {:.4f} RMSE {:.4f}".format(spearmanr(train_pred, y_train_repr)[0], np.sqrt(mean_squared_error(train_pred, y_train_repr))))
    print("Valid | Spearman {:.4f} RMSE {:.4f}".format(spearmanr(valid_pred, y_valid)[0], np.sqrt(mean_squared_error(valid_pred, y_valid))))
    print("ValRe | Spearman {:.4f} RMSE {:.4f}".format(spearmanr(valid_repr_pred, y_valid_repr)[0], np.sqrt(mean_squared_error(valid_repr_pred, y_valid_repr))))

fit!
>> SVR C10
Train | Spearman 0.8553 RMSE 0.6743
Valid | Spearman 0.4785 RMSE 0.9211
ValRe | Spearman 0.3842 RMSE 1.1025
